In [42]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import torch
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sentence_transformers import CrossEncoder

In [17]:
# --- 1. Glossary to Grouping Label ---
df = pd.read_csv("../data/2_glossary_to_label_gt.csv")
df_glossary_desc = pd.read_csv("../data/1b_glossary_descriptions.csv")

# --- 2. Load grouping_master to get all possible grouping_labels ---
grouping_df = pd.read_csv("../data/fbi_grouping_master.csv")
all_labels = grouping_df["grouping_label"].tolist()

In [18]:
# Merge df with df_glossary_desc on the 'Glossary' column
df_merged = df.merge(df_glossary_desc[['Glossary', 'Description']], on='Glossary', how='left')
df_merged.head()

,Glossary,ground_label,grouping_id,Description
0,Assets,Total Assets,5300004,"Resources owned by a company (e.g., cash, inve..."
1,Liabilities,(Current liabilities) + (Non-current liablilit...,"(5300177),(5300146)",Obligations or debts the company must pay (e.g...
2,Equity,Equity,5300007,Owners’ interest in the company (Assets - Liab...
3,Current Assets,Current assets,5300033,Assets expected to be used or converted to cas...
4,Non-Current Assets,Non-current assets,5300031,"Long-term assets (e.g., property, plant, and e..."


In [48]:
# --- 3. Initialize embedding model ---
model = SentenceTransformer('BAAI/bge-large-en-v1.5')

In [55]:
#reranker = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")
reranker = CrossEncoder("../models/stage2_cross_encoder_finetuned")

In [61]:
# --- 4. Embed all grouping_labels once ---
label_embs = model.encode(all_labels, convert_to_tensor=True)

In [62]:
# --- 5. For each predicted glossary term, find best grouping_label ---
predicted_labels = []

for _, row in df_merged.iterrows():
    # Add description to glossary item
    glossary = row['Glossary']
    #description = row['Description']
    #query = glossary + ' can be defined as ' + description
    query = glossary
    #print(query)

    # Embed and match
    q_emb = model.encode(query, convert_to_tensor=True)
    sims = util.cos_sim(q_emb, label_embs)[0]
    #best_idx = torch.argmax(sims).item()
    #predicted_labels.append(all_labels[best_idx])

    # Reranker
    top_k = 5
    top_k_indices = torch.topk(sims, k=top_k).indices.tolist()
    top_k_labels = [all_labels[i] for i in top_k_indices]

    # Step 2: Rerank top-k candidates using cross-encoder
    pairs = [(query, label) for label in top_k_labels]
    rerank_scores = reranker.predict(pairs)

    # Step 3: Pick best reranked label
    best_rerank_idx = torch.tensor(rerank_scores).argmax().item()
    predicted_label = top_k_labels[best_rerank_idx]

    predicted_labels.append(predicted_label)


df_merged["predicted_label"] = predicted_labels

df_merged["predicted_grouping_id"] = [
    grouping_df.loc[grouping_df["grouping_label"] == label, "grouping_id"].iat[0]
    if label in grouping_df["grouping_label"].values else None
    for label in df_merged["predicted_label"]
]

In [64]:
# --- 6. Evaluate at label level ---
y_true = df_merged["ground_label"]
y_pred = df_merged["predicted_label"]

accuracy = accuracy_score(y_true, y_pred)
f1_macro = f1_score(y_true, y_pred, average="macro")
f1_micro = f1_score(y_true, y_pred, average="micro")

print(f"Accuracy (labels): {accuracy:.4f}")
print(f"Macro F1 (labels): {f1_macro:.4f}")
#print(f"Micro F1 (labels): {f1_micro:.4f}")
#print("\nClassification Report (labels):\n")
#print(classification_report(y_true, y_pred, digits=4))

Accuracy (labels): 0.3793
Macro F1 (labels): 0.2370


In [54]:
# --- 7. Save enriched CSV ---
path = "../results/25May_1b/stage2_glossary_to_label.csv"
df_merged.to_csv(path, index=False)
print("✅ Saved " + path)

✅ Saved ../results/25May_1b/stage2_glossary_to_label.csv
